### **Fake News Classifier Using Bidirectional LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/gdrive/My Drive/LSTM/train.csv')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop Nan Values

df=df.dropna()

In [5]:
# Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
## Get the Dependent features

y=df['label']

In [8]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.4.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [15]:
# Vocabulary size

voc_size=5000

#### **Onehot Representation**

In [16]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0:5]

[[1447, 939, 3505, 164, 2525, 888, 4917, 4896, 4369, 293],
 [1797, 589, 2697, 1755, 2696, 3260, 2831],
 [637, 3633, 3676, 1529],
 [3300, 4445, 1065, 4761, 927, 2189],
 [3847, 2696, 353, 920, 3933, 4973, 2696, 1119, 3297, 2490]]

### **Embedding Representation**

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4896 4369  293]
 [   0    0    0 ... 2696 3260 2831]
 [   0    0    0 ... 3633 3676 1529]
 ...
 [   0    0    0 ... 1598 4302 1098]
 [   0    0    0 ...  891 2233  965]
 [   0    0    0 ... 2238  238  918]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1447,
        939, 3505,  164, 2525,  888, 4917, 4896, 4369,  293], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### **Model Training**

In [32]:
# Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 15ms/step - loss: 0.4432 - accuracy: 0.7618 - val_loss: 0.1987 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1382 - accuracy: 0.9498 - val_loss: 0.2029 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0835 - accuracy: 0.9712 - val_loss: 0.2223 - val_accuracy: 0.9150
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0572 - accuracy: 0.9818 - val_loss: 0.2953 - val_accuracy: 0.9125
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0508 - accuracy: 0.9826 - val_loss: 0.3293 - val_accuracy: 0.9099
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0338 - accuracy: 0.9901 - val_loss: 0.4151 - val_accuracy: 0.9079
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0220 - accuracy: 0.9942 - val_loss: 0.4121 - val_accuracy: 0.906

### **Adding Dropout**

You can add this code for dropout

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### **Performance Metrics And Accuracy**

In [33]:
#y_pred=model.predict_classes(X_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3130,  289],
       [ 255, 2361]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9098591549295775

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

